In [3]:
import pandas as pd
import os


# 処理対象のフォルダパス (Windowsパスはr'...'でRaw Stringにすると安全)
PROJECT_PATH = r'C:\Users\robek\OneDrive\ドキュメント\職務経歴書'
# 出力するExcelファイル名
OUTPUT_FILE = 'Project_Document_Breakdown.xlsx'

# ----------------------------------------------------------------------
# 2. generate_visual_tree (データ生成関数)
# ----------------------------------------------------------------------

def generate_visual_tree(startpath):
    """
    指定されたパスのフォルダ構造をスキャンし、
    シート分割用の辞書形式（フォルダ名: ツリー文字列リスト）でデータを返す関数
    """
    
    folder_data = {}
    space = '    '
    pipe_space = '│' + space

    if not os.path.isdir(startpath):
        # 呼び出し元が処理できるように、エラー情報を辞書として返す
        return {'エラー': [f'指定されたフォルダ({startpath})が見つからないか、フォルダではありません。スキップします。']}
    
    root_key = os.path.basename(startpath) + os.sep
    root_lines = []
    root_lines.append(root_key)

    root_items = os.listdir(startpath)
    root_files = [
        item for item in root_items
        if os.path.isfile(os.path.join(startpath, item))
    ]
    root_files.sort()

    has_subdirs = any(os.path.isdir(os.path.join(startpath, item)) for item in root_items)

    for i, file in enumerate(root_files):
        is_last_file = (i == len(root_files) - 1)
        
        if is_last_file and not has_subdirs:
            root_prefix = '└── '
        else:
            root_prefix = '├── '
            
        root_lines.append(f'{root_prefix}📄{file}')
        
    folder_data[root_key] = root_lines
    
    for root, dirs, files in os.walk(startpath):
        if root == startpath:
            for dir_name in dirs:
                folder_data[dir_name + os.sep] = [dir_name + os.sep]
            continue
            
        dirs.sort()
        files.sort()

        root_relative = root.replace(startpath + os.sep, '')
        top_level_dir = root_relative.split(os.sep)[0] + os.sep
        
        if top_level_dir in folder_data:
            current_tree_list = folder_data[top_level_dir]
        else:
            continue
            
        level = root_relative.count(os.sep)
        indent_base = pipe_space * (level - 1)
        dir_name = os.path.basename(root)
        
        is_last_dir_in_list = dir_name == (dirs[-1] if dirs else dir_name)

        if is_last_dir_in_list:
            folder_prefix = '└── '
            next_indent = space
        else:
            folder_prefix = '├── '
            next_indent = pipe_space
        
        current_tree_list.append(f'{indent_base}{folder_prefix}📂{dir_name}{os.sep}')
        
        file_indent_base = indent_base + next_indent

        for i, file in enumerate(files):
            is_last_file = (i == len(files) - 1)
            
            if is_last_file:
                file_prefix = '└── '
            else:
                file_prefix = '├── '
            
            current_tree_list.append(f'{file_indent_base}{file_prefix}📄{file}')

    return folder_data

# ----------------------------------------------------------------------
# 3. write_trees_to_excel (Excel出力関数)
# ----------------------------------------------------------------------

def write_trees_to_excel(folder_data, output_filepath):
    
    print(f'\n📊 Excelファイルへの書き出しを開始します: {output_filepath}')
    
    try:
        with pd.ExcelWriter(output_filepath, engine='xlsxwriter') as writer:
            
            for folder_name, tree_lines in folder_data.items():
                
                if folder_name == 'エラー':
                    print(f"   警告：データにエラー情報が含まれていました。スキップします: {tree_lines[0]}")
                    continue
                    
                df = pd.DataFrame(tree_lines, columns=['Path']) 
                
                sheet_name = folder_name.replace(os.sep, '_').replace('/', '_').replace('\\', '_')
                sheet_name = sheet_name.strip('_')
                
                if not sheet_name: 
                    sheet_name = 'ルート'
                
                sheet_name = sheet_name[:31] 
                
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                
                try:
                    worksheet = writer.sheets[sheet_name]
                    worksheet.set_column(0, 0, 50) 
                except Exception as e:
                    pass

        print(f"✅ Excelファイルが正常に作成されました。")

    except Exception as e:
        print(f"❌ Excelファイル書き出し中に致命的なエラーが発生しました: {e}")

# ----------------------------------------------------------------------
# 4. メイン処理の実行
# ----------------------------------------------------------------------

if __name__ == "__main__":
    # 現場の例え：施工管理。まずデータを集め、次に帳簿に出力する。
    
    print("--- 建築DXツール：フォルダ構造解析開始 ---")
    
    # データを生成
    tree_data = generate_visual_tree(PROJECT_PATH)
    
    # エラーチェック
    if 'エラー' in tree_data:
        print(f"🚨 処理中断：{tree_data['エラー'][0]}")
    else:
        # Excelに出力
        write_trees_to_excel(tree_data, OUTPUT_FILE)
    
    print("--- 処理完了 ---")

--- 建築DXツール：フォルダ構造解析開始 ---

📊 Excelファイルへの書き出しを開始します: Project_Document_Breakdown.xlsx
✅ Excelファイルが正常に作成されました。
--- 処理完了 ---
